Need to modify existing opcode computer: 

First, you'll need to add two new instructions:

* Opcode 3 takes a single integer as input and saves it to the position given by its only parameter. For example, the instruction 3,50 would take an input value and store it at address 50.
* Opcode 4 outputs the value of its only parameter. For example, the instruction 4,50 would output the value at address 50. 

Programs that use these instructions will come with documentation that explains what should be connected to the input and output. The program 3,0,4,0,99 outputs whatever it gets as input, then halts.

Functions to load and parse data:

In [1]:
def load_data(path):
    with open(path) as f:
        program = f.read()
    return program

def parse_data(program): 
    """Function to split string at every comma and return list of strings.
    Some will be converted to int later, but need to start as strings."""
    
    program = program.split(',')
    program = [value for value in program]
    return program

In [2]:
program = parse_data(load_data('data/day_5.txt'))
print(program[:50])

['3', '225', '1', '225', '6', '6', '1100', '1', '238', '225', '104', '0', '1102', '9', '19', '225', '1', '136', '139', '224', '101', '-17', '224', '224', '4', '224', '102', '8', '223', '223', '101', '6', '224', '224', '1', '223', '224', '223', '2', '218', '213', '224', '1001', '224', '-4560', '224', '4', '224', '102', '8']


Have to think carefully here. What lengths can the strings come in? How to identify each one? 

* Code 1 is 4 digits, from right to left: 2-digit code (01), parameter 1, parameter 2 (unstated parameter 3, always 0)
* Code 2 is 4 digits, from right to left: 2-digit code (02), parameter 1, parameter 2 (unstated parameter 3, always 0)
* Code 3 is 3 digits, from right to left: 2-digit code (03), parameter 1
* Code 4 is 3 digits, from right to left: 2-digit code (04), parameter 1
* Code 99 is 2 digits: 99

In [3]:
def parse_opcode(opcode): 
    """Given an opcode string as part of a program, this function
    splits the string to identify: 
    1) The opcode number, 
    2) The modes of the first two parameters. Omitting parameter 3 because 
    it's a write operation and will always be 0 (position mode)."""
    
    full = str(opcode)
    code = int(full[-2:])
    
    if len(full) == 4: 
        mode_1 = int(full[-3])
        mode_2 = int(full[-4])
    
    elif len(full) == 3:
        mode_1 = int(full[-3])
        mode_2 = None
    
    else: # Code is 3 or 99
        mode_1 = None
        mode_2 = None
        
    return (code, mode_1, mode_2)

# Test

print(parse_opcode('0103'))
print(parse_opcode('99'))
print(parse_opcode('03'))

(3, 1, 0)
(99, None, None)
(3, None, None)


New approach: Create sub-function to process a single opcode. Keep the while loop to call it for each opcode, skipping ahead as determined by the pointer. 

In [30]:
def run_opcode(program, index, code, input_val = None):
    """Function to process a single opcode located at the index specified, 
    given a full program and a tuple of code plus parameter modes."""
    
    p = program.copy()
    output = None
    
    if code[0] in [1, 2]:
        param_1_dict = {0: p[int(p[index + 1])], 
                        1: p[index + 1]}
        param_2_dict = {0: p[int(p[index + 2])], 
                        1: p[index + 2]}
    
    if code[0] == 1:
        p[int(p[index + 3])] = str(int(param_1_dict[code[1]]) + int(param_2_dict[code[2]]))
        index += 4
    elif code[0] == 2:
        p[int(p[index + 3])] = str(int(param_1_dict[code[1]]) * int(param_2_dict[code[2]]))
        index += 4
    elif code[0] == 3: 
        p[int(p[index + 1])] = str(input_val)
        index += 2
    elif code[0] == 4:
        output = p[index + 1]
        index += 2
    elif code[0] == 99:
        output = "finished"
    else:
        raise Exception("Not a valid opcode")
        
    return (p, index, output) # In calling function, can retrieve tuple values by index

In [31]:
param_dict = {0: "code 0", 
             1: "code 1"}

code = [0, 1]

print(param_dict[code[0]])

code 0


Run some tests on this function:

In [32]:
test3 = ['1002','4','3','4','33']
print(test3)

['1002', '4', '3', '4', '33']


In [33]:
a = parse_opcode(test3[0])
print(a)

(2, 0, 1)


In [34]:
b = run_opcode(program = test3, index = 0, code = a)
print(b)
type(b)

(['1002', '4', '3', '4', '99'], 4, None)


tuple

In [35]:
test4 = ['03', '2', '0']
c = parse_opcode(test4[0])
d = run_opcode(program = test4, index = 0, code = c, input_val = 5)
print(d)

(['03', '2', '5'], 2, None)


Function to run the entire program:

In [36]:
def run_program_v1(program, input_val): 
    
    p = program.copy()
    pointer = 0
    finished = False
    output = []

    while pointer < len(p) and not finished: 
        #print(pointer)
        code = parse_opcode(p[pointer]) 
        result = run_opcode(p, pointer, code, input_val)
        p = result[0]
        if result[2] == "finished":
            finished = True 
        else: 
            pointer = result[1]
        
    return output

In [37]:
test3 = ['1002','4','3','4','33']
print(test3)

['1002', '4', '3', '4', '33']


In [38]:
k = parse_opcode(test3[0])
print(k)

(2, 0, 1)


In [39]:
print(run_opcode(test3, 0, k))

(['1002', '4', '3', '4', '99'], 4, None)


In [40]:
x = run_program_v1(test3, input_val = 3)
print(x)

[]


That's something to build on.

If necessary, see Ray's answer: https://github.com/raybuhr/adventofcode/commit/ce754779f006b572ddee38fbb14ef055ce2cd017